# Run a behavior cloning behavior

Run a behavior cloning based robot controller on a real robot. 

* Initialize the robot controller and the camera controller, based on the exp
* Load the behavior cloning controller based on the exp, together with the sp
* Initialize a loop which loads an image through the camera, passes it through the sp, the controller and applies the controller output to the robot. 


In [ ]:
# verifying pathlib is clean
import pathlib
print(pathlib.__file__)

In [ ]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pprint
import math
import socket
import pathlib
import time
import torch
import numpy as np

from robot.al5d_position_controller import PositionController, RobotPosition

hostname = socket.gethostname()
print(f"Hostname is {hostname}")
if hostname == "raven":
    # the gamepad is only available on some machines, and only on Linux
    available_gamepad = True
    preferred_camera = "dev0"
    try:
        from remote_control.gamepad_controller import GamepadController
    except ModuleNotFoundError:
        print("Approxend module not found, cannot use gamepad")
        available_gamepad = False
    available_robot = True
elif hostname == "szenes.local":
    available_gamepad = False
    available_robot = False
    preferred_camera = "dev0"
elif hostname == "glassy":
    available_gamepad = False
    available_robot = False
    preferred_camera = "dev0"
else:
    available_gamepad = False
    available_robot = False
    
from remote_control.keyboard_controller import KeyboardController
from remote_control.automove_controller import AutoMoveController

# the robot is not always available, this allows us to run the demonstration 
# without the robot


from camera.camera_controller import CameraController
import sensorprocessing.sp_helper as sp_helper
from sensorprocessing.sp_factory import create_sp
import bc_factory 


Config().runtime["device"] = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# device = "cpu"
print(f"Using device: {Config().runtime['device']}")

In [ ]:
# Setting up a separate directory for generated and computed data
exprun_path, result_path = bc_factory.external_setup("BerryPicker-BC")

In [ ]:
# load the exp/runs
experiment = "behavior_cloning"
runs = Config().list_runs(experiment)
pprint.pprint(runs)
# this is a specialized run, just for running a robot controller, setting up the camera etc. 
# FIXME: probably I will need to generate these as well from flow...
# FIXME: for the time being I had just edited one in the created directory
# run = "bc_run_00"
run = "runbc_bc_lstm_0001"
exp = Config().get_experiment(experiment, run)

# note that we are loading the exprun for the robot controller even if the robot is not connected
exp_robot_controller = Config().get_experiment(exp["exp_robot_controller"], exp["run_robot_controller"])

if hostname == "glassy":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_glassy")
elif hostname == "raven":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_raven")
elif hostname == "szenes.local":
    exp_camera_controller = Config().get_experiment("controllers", "camera_controller_szenes")    
else:
    exp_camera_controller = Config().get_experiment(exp["exp_camera_controller"], exp["run_camera_controller"])

# the behavior cloning exp/run
exp_bc = Config().get_experiment(exp["exp_bc"], exp["run_bc"])
# the sensor processing exp/run
exp_sp = Config().get_experiment(exp_bc["exp_sp"], exp_bc["run_sp"])

In [ ]:
# starting the robot controller
if available_robot:
    robot_controller = PositionController(exp_robot_controller)
else:
    robot_controller = None
# starting the camera controller
camera_controller = CameraController(exp_camera_controller)
camera_controller.visualize = True

# starting the sensor processing
sp = create_sp(exp_sp)

# load the behavior cloning controller
model, _, _ = bc_factory.create_bc_model(exp_bc, exp_sp)
model_path = pathlib.Path(exp_bc.data_dir(), exp_bc["controller_file"])
model.load_state_dict(torch.load(model_path, map_location=Config().runtime["device"]))   

### Running the robot

Run the robot controller in a loop, with a human in the loop test at every move.

In [ ]:
def safe_robot_control(a, robot_controller, exp_robot_controller):
    """Moves the robot to the position, with various safety controler"""
    pos_target = RobotPosition.from_normalized_vector(exp_robot_controller,a)
    print(f"Target position: {pos_target}", flush=True)
    if not robot_controller:
        return "0"
    # we assume that we have the robot
    # ask the user whether we indeed want to continue moving?
    pos_current = robot_controller.get_position()
    distance = pos_current.empirical_distance(exp_robot_controller, pos_target)
    query = f"Next position: {pos_target} at distance={distance} Proceed with the move? (A/Y/N/Q)"
    ans = input(query)

    # create an artificial target, changing the heading, just in case

    if ans.lower() == "y": # move towards the target, a safe amound
        pos_target_safe = RobotPosition(exp_robot_controller, pos_current.values)

        diff = (pos_target["height"] - pos_current["height"])
        diff = min(0.1, max(diff, -0.1))
        pos_target_safe["height"] += diff

        diff = (pos_target["heading"] - pos_current["heading"])
        diff = min(2, max(diff, -2))
        pos_target_safe["heading"] += diff


        robot_controller.move(pos_target_safe)
        ""
        return "y"
    elif ans.lower() == "a": # make an artificial move, just to see that we are controlling the robot
        pos_target_artificial = RobotPosition(exp_robot_controller, pos_current.values)
        pos_target_artificial["heading"] = pos_target_artificial["heading"] + 3
        robot_controller.move(pos_target_artificial)
    elif ans.lower() == "q":
        return "q"


In [ ]:
def run_robot(model, robot_controller, exp_robot_controller, camera_controller, controller_interval = 0.1):
    """Robot running loop. Loads the image from the camera, runs the model, and executes the recommended code."""
    # last_interval = 0
    controller_interval = 0.1

    transform = sp_helper.get_transform_to_sp(exp_sp)
    # the backlog of the last sequence length steps
    z_seq = []

    while True:
        start_time = time.time() 
        # Let us assume that the first image here is the one
        toexit = camera_controller.update()
        if toexit:
            break
        # FIXME: this should be made different... based on the run
        # image = camera_controller.images[exp["control_camera"]]
        active_camera = camera_controller.exp["active_camera_list"][0]
        image = camera_controller.images["dev" + str(active_camera)]
        # FIXME: this is totally won't work like this
        # there should be a notebook with the input processing???
        sensor_readings = sp_helper.load_capture_to_tensor(image, transform)
        z = sp.process(sensor_readings[0])
        print(z)
        if exp_bc["sequence_length"]: # no sequence processsing
            z_seq.append(z)
            if len(z_seq) > exp_bc["sequence_length"]:
                z_seq.pop(0)
            if len(z_seq) < exp_bc["sequence_length"]:
                ## we don't have enough in the queue, wait.
                print("Not acting, waiting on the queue", flush=True)
                z_tensor = None
                pass
            else:
                # FIXME: this should be applied online, not step by step
                z_seq_np = np.array(z_seq)
                z_tensor = torch.tensor(z_seq_np).unsqueeze(0).to(Config().runtime["device"])
        else:
            z_tensor = torch.tensor(z).unsqueeze(0).to(Config().runtime["device"])
        if z_tensor == None:
            continue
        if not model.stochastic:                
            a_pred = model(z_tensor)
        else:
            a_pred = model.forward_and_sample(z_tensor)

        retval = safe_robot_control(a_pred[0], robot_controller, exp_robot_controller)
        if retval == "q":
            break

        # ensuring that the execution happens 
        end_time = time.time() 
        execution_time = end_time - start_time 
        # last_interval = execution_time
        time_to_sleep = max(0.0, controller_interval - execution_time) 
        time.sleep(time_to_sleep) 

In [ ]:
run_robot(model = model, robot_controller = robot_controller, exp_robot_controller = exp_robot_controller, camera_controller = camera_controller)

In [ ]:
# shutdown
camera_controller.stop()
if robot_controller:
    robot_controller.stop_robot()